In [1]:
import geopandas as gpd

In [2]:
gdf = gpd.read_file("s1a_mp_user_20240607t174000_20240627t194000.geojson")
gdf_iw = gdf[gdf['mode'] == "IW"]

In [3]:
gdf

,begin_date,end_date,mode,orbit_absolute,orbit_relative,pol,geometry
0,2024-06-07 18:09:31,2024-06-07 18:12:44,IW,54221,74,DV,"POLYGON ((1.04793 2.03890, -1.36823 13.71677, ..."
1,2024-06-07 18:17:17,2024-06-07 18:25:50,IW,54221,74,DV,"POLYGON ((-4.85819 30.20985, -5.86847 34.65643..."
2,2024-06-07 18:28:11,2024-06-07 18:32:24,IW,54221,74,DH,"POLYGON ((-19.06117 69.22324, -20.04411 70.393..."
3,2024-06-07 18:35:56,2024-06-07 18:36:32,EW,54221,74,DH,"POLYGON ((-150.42047 80.11543, -157.41911 78.3..."
4,2024-06-07 19:05:33,2024-06-07 19:09:30,IW,54221,74,DV,"POLYGON ((156.48318 -25.03819, 153.26910 -35.6..."
...,...,...,...,...,...,...,...
1588,2024-06-27 18:20:24,2024-06-27 18:22:36,IW,54512,15,SH,"POLYGON ((40.47023 -74.13786, 24.71900 -67.799..."
1589,2024-06-27 18:41:55,2024-06-27 18:46:53,IW,54512,15,DV,"POLYGON ((-6.78783 0.17358, -8.00093 5.97537, ..."
1590,2024-06-27 18:49:22,2024-06-27 18:53:35,IW,54513,16,DV,"POLYGON ((-12.42288 27.18453, -15.94338 42.181..."
1591,2024-06-27 18:57:49,2024-06-27 19:00:29,IW,54513,16,DV,"POLYGON ((-20.84242 57.63423, -21.89285 60.103..."


In [4]:
# https://github.com/geopandas/geopandas/issues/2378#issuecomment-1273199122
import folium
import pandas as pd
import numpy as np


def explore(gdf: gpd.GeoDataFrame, *args, column=None, **kwargs) -> folium.Map:
    """Version of `GeoDataFrame.explore()` method that handles columns of 
    DateTime64 somewhat reasonably. All arguments are passed on to `.explore()` after
    handling DateTime64 columns in `gdf` or `column`.
    
    Can be used in a pipe like so: 
        gdf.pipe(explore, m=m, column="datetime64_col", name="time")"""

    dt_columns = gdf.select_dtypes(["datetime64", "timedelta64"]).columns

    if isinstance(column, str):
        column = gdf[column]
    if pd.api.types.is_datetime64_any_dtype(column):
        column = column.astype("int64")
        # this keeps the branca from encountering an overflow in long_scalars
        column = column-column.min() 
        column = column / column.max() 
    
    gdf = gdf[:]
    gdf[dt_columns] = gdf[dt_columns].astype("str")
    return gdf.explore(*args, column=column, **kwargs)

In [6]:
explore(gdf_iw, column='pol', categorical=True)